In [2]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')

In [3]:
files = glob('seoul_data/*.csv')
files

['seoul_data\\sales_summary_data_20214q.csv',
 'seoul_data\\test.csv',
 'seoul_data\\상권_추정매출.csv(2017_2021).csv',
 'seoul_data\\상권배후지_소득소비.csv',
 'seoul_data\\서울_상권_점포_수.csv',
 'seoul_data\\서울_상주배후지_상주인구.csv',
 'seoul_data\\서울특별시_상권배후지_생활인구.csv',
 'seoul_data\\서울특별시_상권배후지_아파트.csv',
 'seoul_data\\서울특별시_상권배후지_직장인구.csv',
 'seoul_data\\서울특별시_상권배후지_집객시설.csv',
 'seoul_data\\서울특별시_행정동별_상권변화지표.csv',
 'seoul_data\\자치구_평당임대료.csv',
 'seoul_data\\자치구별_상권_변화지표.csv']

In [12]:
df_market = pd.read_csv(files[2])
df_lease = pd.read_csv(files[-2], encoding='cp949')

In [13]:
df_market.columns, df_lease.columns

(Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
        '서비스_업종_코드', '서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수', '주중_매출_비율',
        '주말_매출_비율', '월요일_매출_비율', '화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
        '금요일_매출_비율', '토요일_매출_비율', '일요일_매출_비율', '시간대_00~06_매출_비율',
        '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
        '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율',
        '연령대_10_매출_비율', '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율',
        '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
        '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
        '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
        '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
        '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
        '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
        '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수'

In [14]:
# 상권 매출정보와 평당 임대료 df를 넣으면 병합한 데이터 프레임을 준다
# 구별 평당 임대료, 월평균 매출, 객단가까지 함쳐서 준다
def get_dataframe(df_market, df_lease):
    temp_list = []
    
    df_market = df_market[df_market['기준_년_코드'] == 2021]
    df_lease = df_lease.set_index('Unnamed: 0')
    
#    구별 평당 임대료 데이터 넣기
    for gu in df_market['구'].unique():
        df_temp = df_market[df_market['구'] == gu].copy()
        df_temp['평당 임대료']= df_lease.loc[gu, '연평균임대료']
        temp_list.append(df_temp)
    df_temp = pd.concat(temp_list)
    
#     매장 월평균 매출 열 추가하기
    df_temp['매장 월평균 매출'] = df_temp['분기당_매출_금액'] / (df_temp['점포수'] * 3)
    
#     객단가 계산하기
    df_temp['객단가'] = df_temp['분기당_매출_금액'] / df_temp['분기당_매출_건수']
    
    return df_temp
    

In [15]:
df = get_dataframe(df_market, df_lease)
df.columns

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '서비스_업종_코드', '서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수', '주중_매출_비율',
       '주말_매출_비율', '월요일_매출_비율', '화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
       '금요일_매출_비율', '토요일_매출_비율', '일요일_매출_비율', '시간대_00~06_매출_비율',
       '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
       '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율',
       '연령대_10_매출_비율', '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율',
       '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
       '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',

In [16]:
cols = ['기준_분기_코드', '구', '동','서비스_업종_코드_명','분기당_매출_금액', '분기당_매출_건수', '평당 임대료','매장 월평균 매출', '객단가']
df[cols]

,기준_분기_코드,구,동,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,평당 임대료,매장 월평균 매출,객단가
219,1,종로구,평창동,한식음식점,626851391,25502,170391,7.462517e+06,24580.479609
220,1,종로구,평창동,분식전문점,122957138,12039,170391,4.553968e+06,10213.235152
221,1,종로구,평창동,호프-간이주점,5093349,177,170391,5.659277e+05,28775.983051
222,1,종로구,평창동,커피-음료,13507733,1359,170391,9.005155e+05,9939.465048
223,1,종로구,평창동,일반교습학원,37551380,128,170391,4.172376e+06,293370.156250
...,...,...,...,...,...,...,...,...,...
785290,4,강동구,고덕2동,신발,3000000,129,102159,3.333333e+05,23255.813953
785291,4,강동구,고덕2동,시계및귀금속,3601119,136,102159,4.001243e+05,26478.816176
785292,4,강동구,고덕2동,의약품,2142331475,167430,102159,5.950921e+07,12795.385982
785293,4,강동구,고덕2동,문구,65806707,9259,102159,7.311856e+06,7107.323361


In [17]:

def get_sales_lease_top5(service_name, surface_area, df):
    
    df_temp = pd.DataFrame()
#     선택 업종의 데이터만 가져오기
    df_service = df[df['서비스_업종_코드_명'].str.contains(service_name)]
    
#     구별 매출 평균, 평당 임대료 값 가져오기
    for gu in df['구'].unique():
        df_temp.loc[0, gu] = round(df_service[df_service['구'] == gu]['매장 월평균 매출'].mean(), 0)
        df_temp.loc[1, gu] = df_service[df_service['구'] == gu]['평당 임대료'].mean()
        df_temp.loc[2, gu] = round(df_service[df_service['구'] == gu]['객단가'].mean(), 0)
        
#   df_temp 형태 및 열 이름 바꿔주기      
    df_temp = df_temp.T
    df_temp.columns = ['월평균 매출', '평당 임대료', '평균 객단가']
    
#     월평균 매출 - 임대료 값, 임대료 비율 추가하기
    df_temp['매출-임대료'] = df_temp['월평균 매출'] - (df_temp['평당 임대료'] * surface_area)
    df_temp['임대료 비율'] = (df_temp['평당 임대료'] * surface_area) / df_temp['월평균 매출'] * 100
    df_temp = df_temp.sort_values(by='평당 임대료', ascending = True) 
    
#     plt.figure(1)
#     plt.barh(df_temp['매출-임대료'].tail())
#     df_temp['매출-임대료'].tail().plot(kind='barh', 
#                                            title=f'{service_name} 업종 매출-임대료 상위 top5 구')
    return df_temp

In [50]:
# service_name = '한식'
# surface_area = 20
service_name = input('희망하는 업종 선택하시오 : ')
surface_area = int(input('희망하는 평수 입력하시오 : '))
df_3= get_sales_lease_top5(service_name, surface_area, df)
# get_unit_price(service_name, df)
df_3.to_csv('Tableau/매출관련 데이터.csv')

희망하는 업종 선택하시오 : 스포츠
희망하는 평수 입력하시오 : 20


In [51]:
df_3

,월평균 매출,평당 임대료,평균 객단가,매출-임대료,임대료 비율
도봉구,3390868.0,80030.0,227178.0,1790268.0,47.203253
강북구,3734650.0,83980.0,229287.0,2055050.0,44.973425
중랑구,3845056.0,88624.0,234553.0,2072576.0,46.097638
은평구,5754420.0,90899.0,242425.0,3936440.0,31.592758
동대문구,4087053.0,92204.0,236405.0,2242973.0,45.120041
성북구,6298703.0,93350.0,277553.0,4431703.0,29.641023
노원구,5803998.0,98524.0,260415.0,3833518.0,33.950391
광진구,7147067.0,100116.0,254071.0,5144747.0,28.015968
양천구,4625112.0,100299.0,266918.0,2619132.0,43.371490
구로구,4308880.0,101517.0,242043.0,2278540.0,47.119901


In [31]:
df['구'].unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '성북구', '중랑구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [29]:
df.columns

Index(['기준_년_코드', '기준_분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명',
       '서비스_업종_코드', '서비스_업종_코드_명', '분기당_매출_금액', '분기당_매출_건수', '주중_매출_비율',
       '주말_매출_비율', '월요일_매출_비율', '화요일_매출_비율', '수요일_매출_비율', '목요일_매출_비율',
       '금요일_매출_비율', '토요일_매출_비율', '일요일_매출_비율', '시간대_00~06_매출_비율',
       '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
       '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율', '여성_매출_비율',
       '연령대_10_매출_비율', '연령대_20_매출_비율', '연령대_30_매출_비율', '연령대_40_매출_비율',
       '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
       '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',

In [20]:
def get_service_seoul_data(service_name, df):
    df_temp = pd.DataFrame()
    
    #     선택 업종의 데이터만 가져오기
    df_service = df[df['서비스_업종_코드_명'].str.contains(service_name)]
    
    #     해당 업종 2021년 전체 매출, 점포수 합
    df_temp.loc[0, '서울_전체_매출'] = df_service['분기당_매출_금액'].sum()
    df_temp.loc[0, '서울_전체_점포수'] = df_service[df_service['기준_분기_코드'] == 4]['점포수'].sum()
    
    #     2021년 분기별 매출합
    for no in range(1, 5):
        df_temp.loc[0, f'서울_전체_매출_{no}분기'] = df_service[df_service['기준_분기_코드'] == no]['분기당_매출_금액'].sum()
    
    #     주중, 주말 매출합
    df_temp.loc[0, '주중_매출합'] = df_service['주중_매출_금액'].sum()
    df_temp.loc[0, '주말_매출합'] = df_service['주말_매출_금액'].sum()
    
    #     남성, 여성 매출합
    df_temp.loc[0, '남성_매출합'] = df_service['남성_매출_금액'].sum()
    df_temp.loc[0, '여성_매출합'] = df_service['여성_매출_금액'].sum()
    df_temp.loc[0, '남성_객단가'] = df_service['남성_매출_금액'].sum() / df_service['남성_매출_건수'].sum()
    df_temp.loc[0, '여성_객단가'] = df_service['여성_매출_금액'].sum() / df_service['여성_매출_건수'].sum()

    #     요일별 매출 금액
    for day_name in list('월화수목금토일'):
        df_temp.loc[0, f'{day_name}_매출합'] = df_service[f'{day_name}요일_매출_금액'].sum()
    
    #     연령대별 매출 추이
    for no in range(1, 7):
        if no != 6:
            df_temp.loc[0, f'{no}0대_매출합'] = df_service[f'연령대_{no}0_매출_금액'].sum()
        else:
            df_temp.loc[0, f'{no}0대 이상_매출합'] = df_service[f'연령대_{no}0_이상_매출_금액'].sum()
    
    #     연령별 객단가
    for no in range(1, 7):
        if no != 6:
            df_temp.loc[0, f'{no}0대_객단가'] = df_service[f'연령대_{no}0_매출_금액'].sum() / df_service[f'연령대_{no}0_매출_건수'].sum()
        else:
            df_temp.loc[0, f'{no}0대 이상_객단가'] = df_service[f'연령대_{no}0_이상_매출_금액'].sum() / df_service[f'연령대_{no}0_이상_매출_건수'].sum()
    
     #     시간대 매출금액  '시간대_00~06_매출_금액'
    df_temp.loc[0, '00~06_매출합'] = df_service['시간대_00~06_매출_금액'].sum()
    df_temp.loc[0, '06~11_매출합'] = df_service['시간대_06~11_매출_금액'].sum()
    i = 11
    while True:
        
        if i != 17:
            df_temp.loc[0, f'{i}~{i+3}_매출합'] = df_service[f'시간대_{i}~{i+3}_매출_금액'].sum()
            i += 3
            if i >= 24:
                break
        else:
            df_temp.loc[0, f'{i}~{i+4}_매출합'] = df_service[f'시간대_{i}~{i+4}_매출_금액'].sum()
            i += 4
    
    
    return df_temp


In [21]:
pd.set_option('display.max_columns', None)
service_name = '커피'
surface_area = 20

df_2= get_service_seoul_data(service_name, df)
df_2.to_csv('Tableau/기타 정보 데이터.csv')

In [241]:
df_service = df[(df['서비스_업종_코드_명'].str.contains('한식'))& (df['기준_분기_코드'] == 4)]
cols=['서비스_업종_코드_명', '구', '동','기준_분기_코드','점포수', '분기당_매출_금액']
df_service[cols].sum()

서비스_업종_코드_명    한식음식점한식음식점한식음식점한식음식점한식음식점한식음식점한식음식점한식음식점한식음식점한...
구              종로구종로구종로구종로구종로구종로구종로구종로구종로구종로구종로구종로구종로구종로구종로구종...
동              평창동무악동부암동무악동부암동사직동청운효자동부암동평창동사직동청운효자동평창동가회동가회동...
기준_분기_코드                                                    6100
점포수                                                        98600
분기당_매출_금액                                          2934320329016
dtype: object